In [ ]:
import numpy as np
import numpy.ma as ma
import os
import time

import pandas as pd
from lmfit.models import GaussianModel, Model
from sklearn.linear_model import LinearRegression
from sklearn import mixture, metrics

from shapely.geometry import Polygon

from scipy.stats.distributions import norm
from sklearn.neighbors import KernelDensity
from scipy.stats import gaussian_kde

from matplotlib import pyplot as plt
from matplotlib import cm

from skimage.measure import EllipseModel
from matplotlib.patches import Ellipse

from scipy import ndimage as ndi
from scipy.spatial import distance
from numpy import linalg

import skimage
from skimage import io, transform
from skimage.draw import ellipse
from skimage.color import rgb2gray
from skimage import filters
from skimage.morphology import disk
from skimage import color
from skimage import filters, util
from skimage.morphology import disk, skeletonize, ball

from PIL import Image, ImageDraw, ImageFilter, ImageOps
import copy
import cv2
import os
import sys

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.ndimage import distance_transform_edt as edt
from skimage.draw import ellipse
from skimage.measure import find_contours, approximate_polygon, subdivide_polygon
import logging
import glob
from radio_beam.commonbeam import getMinVolEllipse

from src.utils import grainPreprocess, grainShow, grainMark, grainDraw, grainApprox, grainStats, grainMorphology,grainGenerate
from src.cfg import CfgAnglesNames, CfgBeamsNames, CfgDataset

import plotly.graph_objects as go
from matplotlib.image import NonUniformImage

from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl

from tqdm.notebook import tqdm
import json

from sklearn.metrics import pairwise_distances
import seaborn as sns
from matplotlib.colors import LogNorm, Normalize
import scipy
from sklearn import linear_model


## Распределение диаметров

In [ ]:
# pixel = 49.7 / 1000
pixel_1500_1500 = 50 / 983


# images = np.load('data/saved_metadata/images_4x_preprocess.npy')
images = np.load('data/generated_data/images_left_(1500,1500,1)_preprocessed.npy')
# images = np.load('artif.npy')
# images = np.load('images_left_(1500,1500,1)_preprocessed.npy')
# images = np.load('data/saved_np/images_10_mixer_preprocess.npy')

paths = np.load('data/generated_data/metadata_left_(1500,1500,1)_preprocessed.npy')
# paths = np.load('data/saved_np/metadata_10_mixer_preprocess.npy')
paths = [path.replace('\\','/') for path in paths]



# name = 'artif'
name = 'left_(1500,1500,1)'
# name = '10_mixer_preprocess'
# name = '4x_left_(1500,1500,1)'

types_dict = {'Ultra_Co11': 'средние зерна',
              'Ultra_Co25': 'мелкие зерна',
              'Ultra_Co8': 'средне-мелкие зерна',
              'Ultra_Co6_2': 'крупные зерна',
              'Ultra_Co15': 'средне-мелкие зерна'}

# types_dict = {'class_1': 'Type None'}

for step in range(1, 2):
    # print('step: ',step)

    grainGenerate.diametr_approx_save(save_path=name, images=images,paths=paths, types_dict=types_dict, step=step, pixel=pixel_1500_1500,max_images_num_per_class=1)

In [ ]:
def diametr_approx_all(folder,name, step, N, M, indices=None,save=False):
    #
    # вывод распределения длин а- и б- полуосей для разных образцов
    #

    xy_scatter = np.load(f'{folder}/' + CfgBeamsNames.values + f'{name}' + f'{step}.npy', allow_pickle=True)
    xy_linear = np.load(f'{folder}/' + CfgBeamsNames.approx + f'{name}' + f'{step}.npy', allow_pickle=True)
    xy_linear_data = np.load(f'{folder}/' + CfgBeamsNames.approx_data + f'{name}' + f'{step}.npy', allow_pickle=True)
    texts = np.load(f'{folder}/' + CfgBeamsNames.legend + f'{name}' + f'{step}.npy', allow_pickle=True)



    if indices is not None:
        xy_scatter = xy_scatter[indices]
        xy_linear = xy_linear[indices]
        xy_linear_data = xy_linear_data[indices]
        texts = texts[indices]

    plt.rcParams['font.size'] = '20'
    gs = gridspec.GridSpec(1, 2)
    # для статьи 12,18
    plt.figure(figsize=(N, M))

    ax1 = plt.subplot(gs[0])
    ax2 = plt.subplot(gs[1])

    # ax1.set_title('распределение a-полуосей связующего вещества', fontsize=20)
    # ax2.set_title('распределение b-полуосей связующего вещества', fontsize=20)

    ax1.set_ylabel('ln(p(x))', fontsize=15)
    ax1.set_xlabel('длина, мкм', fontsize=15)

    ax2.set_ylabel('ln(p(x))', fontsize=15)
    ax2.set_xlabel('длина, мкм', fontsize=15)

    #  ax3.grid()
    markers = ['v', 's', 'D', 'd', 'p', '*']
    for i in range(len(xy_scatter)):
        ax1.plot(xy_linear[i][0][0], xy_linear[i][0][1])
        ax2.plot(xy_linear[i][1][0], xy_linear[i][1][1])

    ax1.legend(texts[:, 0], fontsize=14, loc=1)
    # print(texts[:, 0])
    ax2.legend(texts[:, 1], fontsize=14, loc=1)
    # print(texts[:, 1])

    for i in range(len(xy_scatter)):
        marker = markers[i]
        ax1.scatter(xy_scatter[i][0][0], xy_scatter[i][0][1], marker=marker)
        ax2.scatter(xy_scatter[i][1][0], xy_scatter[i][1][1], marker=marker)

    if save:
        plt.savefig(f'распределение_полуосей_{name}_шаг_{step}')


    plt.show()


In [ ]:
N = 30
M = 15
# для статье N=10,M=5

step = 2
# save=True
save=False

# folder = '../data/saved np'
# folder = 'data/saved_np'
folder = 'data/saved_metadata'
# name = '10_mixer_preprocess'
name = '4x_left_orig_shape_'
# name = '4x_data_border_fix_'
# name = ''

# types_dict = {'Ultra_Co11': 'средние зерна',
#               'Ultra_Co25': 'мелкие зерна',
#               'Ultra_Co8': 'средне-мелкие зерна',
#               'Ultra_Co6_2': 'крупные зерна',
#               'Ultra_Co15': 'средне-мелкие зерна'}




# diametr_approx_all(folder,name, step, N, M, indices=[0, 2, 3])
diametr_approx_all(folder,name, step, N, M, indices=[3,0,2],save=save)

## Heatmap для двумерного распределения полуосей

In [ ]:
folder = 'data/generated_data//beams'
name = 'left_(1500,1500,1)'
step=2
all_a_beams = np.load(f'{folder}/beams_original_values_a_step_left_(1500,1500,1)2.npy', allow_pickle=True)
all_b_beams = np.load(f'{folder}/beams_original_values_b_step_left_(1500,1500,1)2.npy', allow_pickle=True)

In [ ]:
h_step=1
i=1
pixel = 49.7 / 1000
a_b_zeros=np.zeros((int((max(all_a_beams[i])*pixel))//h_step+1,int((max(all_b_beams[i])*pixel)//h_step+1)))
for j in range(len(all_a_beams[i])):

    key1=all_a_beams[i][j]
    key2=all_b_beams[i][j]

    a_b_zeros[all_a_beams[i][j]//h_step,all_b_beams[i][j]//h_step]+=1

x_sticks=np.arange(a_b_zeros.shape[0]*2,2)
y_sticks=np.arange(a_b_zeros.shape[1]*2,2)

# fig, ax = plt.subplots(figsize=(15,15))
# im = ax.imshow(a_b_zeros)
#
# # ax.set_xticks(np.arange(len(x_sticks)),
# #               x_sticks)
# # ax.set_yticks(np.arange(len(y_sticks)),
# #               labels=y_sticks)
#
# cbar = ax.figure.colorbar(im,
#                           ax = ax,
#                           shrink=0.5 )
# # ax.set_title("Matplotlib Heatmap with imshow", size=20)
# fig.tight_layout()

x=np.array(all_a_beams[i]).reshape((-1))


fig = go.Figure(go.Histogram2d(
    x=all_a_beams[i],
    y=all_b_beams[i]
))
fig.show()

In [ ]:
{'path':paths[i],
 'name':name,
 'type':types_dict[names[i]],
 'legend':[{'a_beams':legend1,'b_beams':legend2}],
 'density_curve_scatter':[{'a_beams':(x1.flatten(),y1.flatten()),'b_beams':(x2.flatten(),y2.flatten())}],
 'linear_approx_plot':[{'a_beams':(x_pred1.flatten(), y_pred1.flatten()),'b_beams':(x_pred2.flatten(), y_pred2.flatten())}],
 'linear_approx_data':[{'a_beams':{'k':k1,'b':b1,'angle':angle1,'score':score1},'b_beams':{'k':k2,'b':b2,'angle':angle2,'score':score2}}],
 'beams_length_series':[{'a_beams':all_a_beams,'b_beams':all_b_beams}],
 'pixel2meter':pixel,
 }

In [ ]:
data['data'][i]['beams_length_series'][0]['a_beams']

In [ ]:
x = np.array(data['data'][i]['beams_length_series'][0]['a_beams'])
y= np.array(data['data'][i]['beams_length_series'][0]['b_beams'])

# x = [6,7,8,14,12,6,5,3,4]
# y = [2,1,3,2,9,3,2,1,1]

bins = range(0,30)

# values, _, _ = np.histogram2d(x,y, bins=bins)

h,_, _, image = plt.hist2d(x,y,bins=bins)
plt.imshow(h)
plt.colorbar(image)

plt.plot(range(50),range(50),color='black')
plt.xlim(0,20)
plt.ylim(0,20)

plt.show()

In [ ]:
matrices=[]
step=2
data = open(f'left_(1500,1500,1)_step_{step}_beams.json',encoding='utf-8')
data = json.load(data)

ellipses_num=[]

for  i in range(5):
    b_beams = np.array(data['data'][i]['beams_length_series'][0]['a_beams'])
    a_beams= np.array(data['data'][i]['beams_length_series'][0]['b_beams'])
    ellipses_num.append(len(a_beams))

    matrix,_, _, image = plt.hist2d(a_beams,b_beams,bins=range(0,32))
    # matrix/=matrix.max()
    matrices.append(matrix)

In [ ]:
cm = plt.cm.get_cmap('RdYlBu_r')

fig=plt.figure(figsize=(25,15))
# fig=plt.figure(figsize=(25,4))

ax1 = fig.add_subplot(151)
ax2 = fig.add_subplot(152)
ax3 = fig.add_subplot(153)
ax4 = fig.add_subplot(154)
ax5 = fig.add_subplot(155)



names=['Ultra_Co11 - средние зерна',
       'Ultra_Co15 - средне-мелкие зерна',
       'Ultra_Co25 - мелкие зерна',
       'Ultra_Co6_2 - крупные зерна',
       'Ultra_Co8 - средне-мелкие зерна']

ims=[]
axes=[ax1,ax2,ax3,ax4,ax5]

for i,ax in enumerate(axes):
    im=ax.imshow(matrices[i], interpolation=None,norm=mpl.colors.LogNorm(), cmap=plt.cm.RdYlBu_r)
    # im=ax.pcolormesh(matrices[i],norm=mpl.colors.LogNorm(), cmap=plt.cm.RdYlBu_r)
    ims.append(im)

for i,ax in enumerate(axes):
    x=np.array(range(30))
    y=[]
    for x_i in x:
        max_val=matrices[i][x_i].max()
        y.append(np.where(matrices[i][x_i]==max_val)[0][0])
    y=np.array(y)

    (x_pred, y_pred), k, b, angle, score=grainApprox.lin_regr_approx(x.reshape((-1,1)),y.reshape((-1,1)))
    ax.plot(y_pred,x_pred,label=f'k={k}',color='green')
    ax.legend([f'k={round(k,2)}\nellipses num={ellipses_num[i]}'])
    ax.scatter(y,x)

big_im=copy.copy(ims[0])

masked_data=big_im.__dict__['_A'].data
for im in ims[1:]:
    masked_data+= im.__dict__['_A'].data

masked_data/=5
new_bg_im=ma.masked_array(masked_data,mask=False)
big_im.__dict__['_A']=new_bg_im

for i,ax in enumerate(axes):
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    # cbar = fig.colorbar(ims[i], cax=cax, orientation='vertical')
    cbar = fig.colorbar(big_im, cax=cax, orientation='vertical')
    cbar.set_label('Количество точек',size=15)

    ax.set_xlabel('Малая полуось, мкм',fontsize=15)
    ax.set_ylabel('Большая полуось, мкм',fontsize=15)
    # ax.invert_yaxis()
    ax.set_title(names[i],fontsize=15)
    ax.set_xlim(0,30)
    ax.set_ylim(0,30)
    ax.grid(False)
    ax.set_facecolor("White")

# cax = plt.axes([0.8, 0.575, 0.035, 0.3])
# plt.colorbar(cax=cax)

# cbar.set_label('Количество точек',size=15)
# ax1.set_ylabel('Большая полуось, мкм',fontsize=15)

fig.subplots_adjust(wspace=0.5)
plt.savefig(f'двумерное_распределение_полуосей_шаг_{step}.png',bbox_inches='tight')
plt.show()

names_new=['AB_Co11_medium',
       'AB_Co15_medium_small',
       'AB_Co25_small',
       'AB_Co6_large',
       'AB_Co_8_medium_small']

img_plot=io.imread(f'двумерное_распределение_полуосей_шаг_{step}.png')
for i in range(len(ims)):
    step_img=img_plot.shape[1]/len(ims)
    io.imsave(f'{names_new[i]}_step_{step}.png',img_plot[:,int(i*step_img):int((i+1)*step_img)])

### Вывод на снимке ненормалных углов

In [ ]:
def draw_edges(image, tol=3, border_eps=5):
    #
    # рисует на изображении линии по точкам контура cnts
    # линии в стиле x^1->x^2,x^2->x^3 и тд
    # на вход подавать обработанное изображение
    #
    new_image = image[:, :, 0]
    im = Image.fromarray(np.uint8(new_image)).convert('RGB')
    draw = ImageDraw.Draw(im)

    cnts = grainMark.get_row_contours(image)
    angles = []

    for j, cnt in enumerate(cnts):
        rules = [image.shape[0] - cnt[:, 0].max() > border_eps, cnt[:, 0].min() > border_eps,
                 image.shape[1] - cnt[:, 1].max() > border_eps, cnt[:, 1].min() > border_eps]
        # проверяем находится ли контур у границы, но это срабатывает очень редко
        if all(rules):
            cnt_aprox = approximate_polygon(cnt, tolerance=tol)
            # основная проверка на неправильные контуры
            if len(cnt_aprox) > 3:
                for i, point in enumerate(cnt_aprox[:-1]):

                    y1, x1 = cnt_aprox[i - 1]
                    y2, x2 = cnt_aprox[i]
                    y3, x3 = cnt_aprox[i + 1]

                    v1 = np.array((x1 - x2, y1 - y2)).reshape(1, 2)
                    v2 = np.array((x3 - x2, y3 - y2)).reshape(1, 2)

                    dot = np.dot(v1[0], v2[0])
                    dist1 = np.linalg.norm(v1[0])
                    dist2 = np.linalg.norm(v2[0])
                    cos = dot / (dist1 * dist2)

                    v = np.concatenate([v1, v2])
                    det = np.linalg.det(v)

                    if abs(cos) < 1:
                        ang = int(np.arccos(cos) * 180 / np.pi)

                        if det < 0:
                            angles.append(ang)
                        else:
                            angles.append(360 - ang)
                    else:
                        if det < 0:
                            angles.append(360)
                            ang = 360
                        else:
                            angles.append(0)
                            ang = 0

                    line_color = (0, 255, 127)

                    if ang <= 30:
                        color = (0, 0, 255)
                        r = 4
                        draw.ellipse((y2 - r, x2 - r, y2 + r, x2 + r), fill=color, width=5)
                        draw.line((y1, x1, y2, x2), fill=line_color, width=4)
                        draw.line((y3, x3, y2, x2), fill=line_color, width=4)

                    elif ang >= 30:
                        color = (255, 0, 0)
                        r = 4
                        draw.ellipse((y2 - r, x2 - r, y2 + r, x2 + r), fill=color, width=5)
                        # draw.line((y1, x1, y2, x2), fill=line_color, width=4)
                        # draw.line((y3, x3, y2, x2), fill=line_color, width=4)


        else:
            continue

    img = np.array(im)

    return img, angles

In [ ]:
# img=io.imread('../data/dataset_4x/Ultra_Co8/Ultra_Co8-001_1.jpeg')
i = 0
img = np.load('data/artif.npy')[1, i]
tol = 3
border_eps = 5
im, angles = draw_edges(img, tol, border_eps)
io.imsave(f'low_angles_artif.png', im)
grainShow.img_show(im, 10, cmap='gray')
